## Étapes

Ce manuel présente comment utiliser la nouvelle recette de personnalisation de l'utilisateur (aws-user-personalization). Cette recette équilibre les recommandations entre les nouveaux et les anciens articles, en vous permettant d'ajuster l'équilibre en faveur des articles plus nouveaux ou plus anciens

De façon générale, l'utilisation de la nouvelle recette de personnalisation de l'UTILISATEUR implique les étapes suivantes :  

1. Configuration d'un client Amazon Personalize
2. Création d'un groupe de données, définition d'un schéma, importation de jeux de données et intégration des interactions en temps réel
4. Création d'une campagne dotée d'une nouvelle configuration `campaignConfig`
5. Création d'un traqueur d'événements pour intégrer les événements envoyés par PutEvents.
6. Appel GetRecommendations, un nouveau champ `RecommendationId` est renvoyé dans la réponse.
7. Appel putEvents avec un `RecommendationId` ou une liste personnalisée d'articles d'impression.
8. Attendez la mise à jour de la campagne.
9. Mettez à jour la campagne manuellement pour arrêter la mise à jour automatique.
10. Nettoyer


> **REMARQUE :** **L'exécution de ce manuel prendra quelques heures.**

### Configuration de la clé d'accès/clé secrète pour l'accès à l'interface API d'AWS.

Assurez-vous que les clés accessKey et secretKey que vous utilisez disposent des autorisations appropriées. Choisissez également la région dans laquelle vous souhaitez exécuter cette démo

In [ ]:
accessKeyId = ""
secretAccessKey = ""
region_name = ""

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError
import time
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [ ]:
suffix = str(np.random.uniform())[4:9]
prefix = 'user-personalization-'
print('prefix+suffix:{}{}'.format(prefix, suffix))
s3_bucket_name = (prefix + suffix).lower()
interaction_schema_name = prefix + 'interaction-'  + suffix
item_metadata_schema_name = prefix + 'items-'  + suffix
dataset_group_name = prefix + suffix
interaction_dataset_name = prefix + 'interactions-' + suffix
item_metadata_dataset_name = prefix + 'items-' + suffix
event_tracker_name = prefix + suffix
solution_name = prefix + suffix
event_tracker_name = prefix + suffix
campaign_name = prefix + suffix

### 1. Configuration du client
Configurons d'abord le client destiné à être personnalisé et Amazon S3.

In [ ]:
# Public s3 bucket owned by Personalize service which used to store the example dataset.

personalize_s3_bucket = "personalize-cli-json-models"
s3_client = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('s3')



####  Initialisation des clients personnalisés

In [ ]:

personalize = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize')
personalize_runtime = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-runtime')
personalize_events = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-events')

### Échantillon de jeu de données

Par commodité et à des fins de démonstration, nous utiliserons l'échantillon de jeu de données fourni par Personalize.
Nous avons fourni deux groupes de données, l'un est constitué de métadonnées d'articles et l'autre de jeu de données d'interactions Commençons par le télécharger en local

In [ ]:
interaction_dataset_key = "sample-dataset/interactions-sample.csv"
items_dataset_key = "sample-dataset/items-with-creation-timestamp-sample.csv"
interactions_file = os.getcwd() + "/interaction_raw.csv"
items_metadata_file = os.getcwd() + "/items_raw.csv"
s3_client.download_file(personalize_s3_bucket, interaction_dataset_key, interactions_file)
s3_client.download_file(personalize_s3_bucket, items_dataset_key, items_metadata_file)

In [ ]:
interactions_df = pd.read_csv(interactions_file)
items_df = pd.read_csv(items_metadata_file)

#### Jetons un coup d'œil au cadre de données sur les interactions.

In [ ]:
interactions_df.head(2)

#### Jeu de données d'interactions
**ITEM_ID**: Article correspondant à l'EVENT_TYPE

**EVENT_TYPE** : Type d'évènement  

**TIMESTAMP** : Horodatage de l'interaction en millisecondes. (Notez que c'est en millisecondes) 

**USER_ID** : Identifiant de l'utilisateur correspondant à cette impression.  

**IMPRESSION :**  À présent, vous pouvez éventuellement transmettre les données d'impression avec les données d'événement dans le jeu de données d'interaction. Cela a lieu dans un nouveau champ, `IMPRESSION`comme vous pouvez le voir ci-dessus, qui prend une concaténation échelonnée des articles avec lesquels l'utilisateur a interagi (par exemple les articles qui ont été présentés à l'utilisateur), les impressions comprennent également les articles sélectionnés.  


In [ ]:
items_df.head(2)

#### Jeu de données d'articles

Les jeux de données d'articles comprennent des ID d'article et des métadonnées associées

**ITEM_ID**: Id d'article  

**genres** Métadonnées de l'article, si plusieurs catégories de classification existent pour le même article. Utilisez dans ce cas un pipe " | " pour concaténer.

**creation_timestamp** Horodatage de l'ajout de l'article

Vous pouvez trouver plus de détails concernant le jeu de données dans la documentation
https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

#### Mise à jour de l'horodatage

Ici, nous allons procéder à la mise à jour de la marque de notre jeu de données fixée à 8 jours à compter d'aujourd'hui afin de montrer l'impact des interactions en temps réel sur nos recommandations.

In [ ]:
current_time = int(time.time())
one_hour_ago = current_time - 8 * 24 * 60 * 60
# Get the time gap between the latest timestamp in the interaction and the current time 
interactions_df = interactions_df.astype({"TIMESTAMP": 'int64'})
latest_time_in_csv = interactions_df["TIMESTAMP"].max()
delta = one_hour_ago - latest_time_in_csv

In [ ]:
# shift the latest timestamp in the interactions_df to be the last hour timestamp
interactions_df.TIMESTAMP = interactions_df.TIMESTAMP + delta
interactions_df.to_csv(os.getcwd() + "/interaction.csv", index = False)

# shift the latest timestamp in the items_df to be the last hour timestamp
items_df = items_df.astype({"creation_timestamp": 'int64'})
items_df.creation_timestamp = items_df.creation_timestamp + delta
items_df.to_csv(os.getcwd() + "/items.csv", index = False)

#### Après la mise à jour, vérifions une fois de plus l'interaction et le jeu de données de l'article.

In [ ]:
interactions_df.head(2)

In [ ]:
items_df.head(2)


### 2. Intégration des données dans Amazon Personalize
Maintenant, créons un groupe de données et un schéma, téléchargeons les données et créons un travail d'importation de données. Cela n'a pas changé.

#### a. Création d'un groupe de données

Cette recette est semblable à la recette existante. La documentation complète se trouve [ici](https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html)

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']

In [ ]:

print('dataset_group_arn : {}'.format(dataset_group_arn))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

#### b. Création des schémas de jeu de données

> **_REMARQUE_** : `Impression` un champ qui dispose d'un type de chaîne et utilise une concaténation échelonnée pour plusieurs valeurs.

In [ ]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "IMPRESSION",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
    ],
    "version": "1.0"
}

In [ ]:
interaction_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)
# print(json.dumps(create_schema_response, indent=2))
interaction_schema_arn = interaction_schema_response['schemaArn']
print('interaction_schema_arn:\n', interaction_schema_arn)

In [ ]:
item_metadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRES",
        "type": "string",
        "categorical": True
    },
    {
        "name": "CREATION_TIMESTAMP",
        "type": "long"
    }
    ],
    "version": "1.0"
}

item_metadata_schema_response = personalize.create_schema(
    name = item_metadata_schema_name,
    schema = json.dumps(item_metadata_schema)
)

# print(json.dumps(create_schema_response, indent=2))
item_metadata_schema_arn = item_metadata_schema_response['schemaArn']
print('item_metadata_schema_arn:\n', item_metadata_schema_arn)

#### c. Création des jeux de données

In [ ]:
interactions_dataset_response = personalize.create_dataset(
    datasetType = 'INTERACTIONS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn,
    name = interaction_dataset_name
)
interaction_dataset_arn = interactions_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('interaction_dataset_arn:\n', interaction_dataset_arn)

items_dataset_response = personalize.create_dataset(
    datasetType = 'ITEMS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_metadata_schema_arn,
    name = item_metadata_dataset_name
)
item_metadata_dataset_arn = items_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('item_metadata_dataset_arn:\n', item_metadata_dataset_arn)

#### d. Téléchargement des jeux de données vers le compartiment S3, configuration de la politique appropriée du compartiment S3, le rôle IAM, etc.,

Nous devons télécharger ces jeux de données ou bien vous pouvez nous fournir le nom du compartiment où vous avez déjà stocké les jeux de données.

In [ ]:
#!aws s3 mb s3://{s3_bucket_name}
s3_bucket_name

In [ ]:
s3_client.create_bucket(Bucket=s3_bucket_name,
                          CreateBucketConfiguration={
                              'LocationConstraint': region_name})

In [ ]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [ ]:
s3_client.upload_file(Filename=interactions_file, Bucket=s3_bucket_name,
    Key="interaction.csv")
s3_client.upload_file(Filename=items_metadata_file, Bucket=s3_bucket_name,
    Key="items.csv")

#### e. Liaison de la politique à votre compartiment S3

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(s3_bucket_name),
                "arn:aws:s3:::{}/*".format(s3_bucket_name)
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=s3_bucket_name, Policy=json.dumps(policy));

#### f. Configuration du rôle IAM approprié pour que la personnalisation puisse accéder aux jeux de données.

In [ ]:
iam = boto3.client(service_name='iam', 
                         aws_access_key_id = accessKeyId, 
                         aws_secret_access_key = secretAccessKey)   


role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

In [ ]:
print('role_arn:', role_arn)

#### g. Create DatasetImportJobs pour télécharger les données

In [ ]:
time.sleep(20) # wait for RoleARN completion
interactions_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'interactions-dij-' + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'interaction.csv')
    },
    roleArn = role_arn
)

interactions_dij_arn = interactions_dij_response['datasetImportJobArn']
print('interactions_dij_arn: ', interactions_dij_arn)
#print(json.dumps(interactions_dij_arn, indent=2))

items_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'items-dij-' + suffix,
    datasetArn = item_metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'items.csv')
    },
    roleArn = role_arn
)

items_dij_arn = items_dij_response['datasetImportJobArn']
print('items_dij_arn:', items_dij_arn)

In [ ]:
dataset_job_arns = [interactions_dij_arn, items_dij_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and len(dataset_job_arns) != 0:
    time.sleep(60)    
    for dij_arn in dataset_job_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dij_arn
        )
        dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
        status = None
        if "latestDatasetImportJobRun" not in dataset_import_job:
            status = dataset_import_job["status"]
            print("{} : {}".format(dij_arn, status))
        else:
            status = dataset_import_job["latestDatasetImportJobRun"]["status"]
            print("DIJ_ARN: {}, LatestDatasetImportJobRun: {}".format(dij_arn, status))
    
        if status == "ACTIVE" or status == "CREATE FAILED":
            dataset_job_arns.remove(dij_arn)


### 3. Create Solution, SolutionVersion

Nous créerons une solution avec 'aws-user-personalization' Cette recette permet d'équilibrer les recommandations des nouveaux et anciens articles fournies aux utilisateurs.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"
max_time = time.time() + 3*60*60 # 3 hours
create_solution_response = None
while time.time() < max_time:

    try:
        create_solution_response = personalize.create_solution(name=solution_name, 
                                    recipeArn= recipe_arn, 
                                    datasetGroupArn = dataset_group_arn)

        solution_arn = create_solution_response['solutionArn']
        print('solution_arn: ', solution_arn)
        break;
    except personalize.exceptions.ClientError as e:
        if 'EVENT_INTERACTIONS' not in str(e):
            print(json.dumps(create_solution_response, indent=2))
            print(e)
            break

#### Création de la SolutionVersion

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 4. Création d'une campagne
Lors de la création de la campagne, nous pouvons définir itemExplorationConfig afin de configurer le poids de l'exploration des articles froids ainsi que l'âge limite de l'exploration. Pour l'instant, nous pouvons définir explorationWeight sur 0,9 et explorationItemAgeCutOff sur 7. Ainsi, nous pensons que tous les articles créés depuis moins de 7 jours seront considérés comme des articles froids, et nous ferons plus d'exploration sur les nouveaux articles.

#### Créer une campagne

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = prefix + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.9",
            "explorationItemAgeCutOff": "7"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
describe_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)
campaign_summary = describe_campaign_response["campaign"]
campaign_summary

### 5. Appeler GetRecommendations
À des fins de démonstration, nous utiliserons les ID utilisateur dans le jeu de données d'entrée pour lancer les appels getRecommendation. 
> **_REMARQUE_** : Dans la réponse, vous avez un nouveau champ `RecommendationId` qui correspond à la liste des articles retournés par Personalize GetRecommendations. Vous pouvez transmettre cet id de recommandation pour indiquer les impressions.  
Vous pouvez également transmettre l'impression sous la forme d'une concaténation de chaînes d'articles. Si vous transmettez à la fois l'id de recommandation et la liste d'impression, la liste d'impression sera prioritaire et sera utilisée par le système.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')
print(rec_response['recommendationId'])

In [ ]:
rec_response['itemList']

### 6. Create Event Tracker

Crée un traqueur d'événement que vous pourrez utiliser lorsque vous envoyez des données d'événement au groupe de données spécifié grâce à l'interface API PutEvents.

In [ ]:
even_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = even_tracker_response['eventTrackerArn']
event_tracking_id = even_tracker_response['trackingId']
#print(json.dumps(even_tracker_response,indent=2))
print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))

### 7. Envoi des données d'impression à Personalize par le biais de PutEvents.
Amazon Personalize peut modéliser deux types d'impressions :  
1. Impressions implicites et impressions explicites. Les impressions implicites sont des impressions qui se produisent pendant la session d'un utilisateur et qui sont automatiquement enregistrées par Amazon Personalize chaque fois qu'un article est présenté à l'utilisateur. Vous pouvez les intégrer dans votre flux de travail de recommandation en incluant l'id de recommandation (retourné par les services et les opérations) comme entrée pour les futures demandes adressées à PutEvents.  


2. Les impressions explicites sont des impressions que vous saisissez manuellement lorsque vous adressez une demande à PutEvents. Vous utiliserez des impressions explicites par exemple lorsque vous ne présenterez pas quelques-uns des articles retournés par GetRecommendations en raison de leur indisponibilité, etc.,   

> **REMARQUE :** Si vous avez défini `impression` dans votre schéma d'interaction comme ci-dessus, vous devez envoyer la liste des impressions (soit les articles renvoyés par GetRecommendations, soit vos propres articles).  
**Lorsque l'ID de recommandation et les impressions sont , Amazon Personalize utilisera par défaut les impressions explicites.**



#### Mettons l'article précédemment recommandé comme impression.

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
     'sentAt': datetime.now().timestamp(),
     'eventType' : 'click',
     'itemId' : rec_response['itemList'][0]['itemId'],        
     'recommendationId': rec_response['recommendationId'],
     'impression': [item['itemId'] for item in rec_response['itemList']],
     }]
    )

#### Nous pouvons également ajouter de nouveaux articles

Mettons le nouvel id de l'article '2xx' dans la personnalisation.

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '240',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '241',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '242',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '243',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '244',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '245',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '246',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '247',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '248',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '249',
     }]
    )

### 8. Création de la nouvelle SolutionVersion grâce à updateMode

Après put-events, veuillez patientez environ 15 minutes pour que Personalize intègre les nouvelles données, après quoi, créez une nouvelle solutionVersion grâce à update-mode

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn, trainingMode = "UPDATE")

solution_version_after_update = create_solution_version_response['solutionVersionArn']
print('solution_version_after_update:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_after_update
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 9. Update Campaign

Mise à jour de la campagne grâce à la dernière solution-version arn issue de la mise à jour.

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, solutionVersionArn=solution_version_after_update)
print('campaign_arn_response: ', campaign_arn_response)

In [ ]:
# Wait for campaign update to reflect the new solution-version
solutionVersionArn = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    solutionVersionArn = describe_campaign_response["campaign"]["solutionVersionArn"]
    print("Campaign solution version: {}".format(solutionVersionArn))
    
    if solutionVersionArn == solution_version_after_update:
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']["solutionVersionArn"]
desc_campaign_response

### Après la mise à jour de la version de la solution, renouvelons la recommandation

Nous nous attendons à ce que de nouveaux articles figurent dans la recommandation, puisque nous avons établi l'exploration supérieure à 0,9

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response['itemList']

### Téléchargement de la campagne avec un poids d'exploration différent

Nous nous attendons à ce que plus d'anciens articles figurent dans la recommandation, puisque nous avons défini un poids d'exploration faible.

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']
desc_campaign_response

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.1",
            "explorationItemAgeCutOff": "7"
        }
    })


In [ ]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.1":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

### Après le téléchargement du poids d'exploration

Renouvelons la recommandation, nous devrions voir plus d'anciens articles ici.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response

#### 10. Suppression des ressources

Après la création de toutes les ressources, supprimons toutes les ressources

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)
    
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)